In [5]:
import numpy as np
import uproot
%matplotlib inline
from coffea import hist
#from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import cloudpickle
import gzip
import time
import os
import glob
from yahist import Hist1D, Hist2D       

In [2]:
pip install yahist

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     |████████████████████████████████| 312 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 7.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=aa23674586a00c97715038cc2c452917c560c13ccc16f6d8b7e4cca119c903d3
  Stored in directory: /home/jovyan/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [22]:
class Processor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "") 
        pt_axis = hist.Bin("pt", " [GeV]", [10., 15., 20., 25., 35., 50., 70., 90.])
        eta_mu_axis= hist.Bin("eta", "", [0., 1.2, 2.1, 2.4])
        eta_el_axis= hist.Bin("eta", "", [0., 0.8, 1.479, 2.5])
        
        self._accumulator = processor.dict_accumulator({
            'Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Tight_Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis),
            'Tight_Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis)

        })
    
    @property
    def accumulator(self):
        
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        dataset = events.metadata["dataset"]
        
        # Skim Muon, Electron, and Jet
        Muon = events.Muon
        Muon = Muon[( Muon.pt >=10 ) &
                           (np.abs(Muon.eta) <= 2.4) &
                           (np.abs(Muon.dxy) <= 0.05) &
                           (np.abs(Muon.dz) <= 0.1) &
                           (np.abs(Muon.sip3d) < 4) &
                           (Muon.looseId==1) &
                           (Muon.ptErr/Muon.pt < 0.2) &
                           (Muon.mediumId==1) ]
        
        Electron = events.Electron
        Electron = Electron[( Electron.pt >10 ) &  #  (Electron.isTriggerSafeNolso) 
                           (np.abs(Electron.eta+Electron.deltaEtaSC ) < 2.4) &
                            (Electron.convVeto)&
                            (Electron.lostHits==0) &
                            (Electron.tightCharge==2) &
                            (np.abs(Electron.dz) < 0.1) &
                           (np.abs(Electron.dxy) < 0.05) &
                           (np.abs(Electron.sip3d) < 4) ]
        Jet = events.Jet
        Jet = Jet[( Jet.pt >25 ) &  (np.abs(Jet.eta ) < 2.4) ]

        
        def delta_phi(first, second):
            return np.arccos(np.cos(first.phi - second.phi))

        def delta_r2(first, second):
            return (first.eta - second.eta) ** 2 + delta_phi(first, second) ** 2

        def match(first, second, deltaRCut=0.4):
            drCut2 = deltaRCut**2
            combs = ak.cartesian([first, second], nested=True)
            return ak.any((delta_r2(combs['0'], combs['1'])<drCut2), axis=2)    
        
        def mt(pt1, phi1, pt2, phi2):    #transverse mass
            return np.sqrt( 2*pt1*pt2 * (1 - np.cos(phi1-phi2)) )
        
        # Definition of Tight/Loose Muon and Electron
        Tight_Muon = Muon[( Muon.pt >20 ) &
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.tightId==1) &
                          (Muon.miniPFRelIso_all<0.16)
                         ]
        Tight_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                       #  (Electron.tightId==1) &
                          (Electron.miniPFRelIso_all<0.12)
                         ]
        
        Loose_Muon = Muon[( Muon.pt >20 )&
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.looseId==1) &
                          (Muon.miniPFRelIso_all<0.4)
                         ]        
        
        Loose_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                        # (Electron.looseId==1) &
                          (Electron.miniPFRelIso_all<0.4)
                         ]
        MET = events.MET   
        

        control_region_Muon = events[( ak.num(Loose_Muon)==1) &(ak.num(Loose_Electron)==0)
                                    & (ak.num(Jet[~match(Jet, Muon, deltaRCut=1.0)])>=1) & ( MET.pt<20) 
                                    & ak.any(mt(Muon.pt,Muon.phi, MET.pt, MET.phi)<20, axis=1)]

        
        tight_muon= control_region_Muon.Muon[(control_region_Muon.Muon.pt >20 ) &
                          (np.abs(control_region_Muon.Muon.eta) < 2.4) &
                         (control_region_Muon.Muon.tightId==1) &
                          (control_region_Muon.Muon.miniPFRelIso_all<0.16)
                         ]
        output['Tight_Muon'].fill(dataset=dataset, pt=ak.flatten(tight_muon.pt), eta=ak.flatten(tight_muon.eta))
        output['Muon'].fill(dataset=dataset, pt=ak.flatten(control_region_Muon.Muon.pt), eta=ak.flatten(control_region_Muon.Muon.eta))
        return output
    def postprocess(self, accumulator):
        return accumulator

### Getting Samples

The folder QCD containes a file `Samples.txt` with all desired samples, and a script `query.sh`. With the command `./query.sh` (with executing permission), one can generate text files containing all the root file names corresponding to the samples in `Samples.txt` using [dasgoclient](https://github.com/dmwm/dasgoclient).

Note that you need your grid passphrase for setting up the proxy.

In [6]:
Samples = glob.glob("QCD/QCD*.txt")

In [19]:
fileset = {}
for n in Samples:
    name = n.split("/")[1].strip(".txt")
    file = open(n,mode='r')
    content = file.read()
    file.close()
    lst = content.split("\n")
    if lst[-1]=='':
        lst = lst[:-1]
    lst = ["root://xcache/" + s for s in lst]  
    fileset[name]=lst

In [20]:
fileset

{'QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8': ['root://xcache//store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/60000/009CB2DA-F651-EB41-89B7-C9C5E69EA963.root',
  'root://xcache//store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/60000/DD5793BF-3D76-4C4C-A21E-F2B296568B53.root',
  'root://xcache//store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/60000/E8DD6408-E093-F345-808B-5F761D8ACDCA.root',
  'root://xcache//store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/60000/E3319065-0F86-8740-9328-A084D259E389.root',
  'root://xcache//store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/

In [ ]:
#fileset = {'SingleMu' : ["root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/60000/30AED9F5-92AD-DD46-A547-61765FD6B596.root"]}
#fileset = {'SingleMu' :QCD}

from dask.distributed import Client
client = Client("tls://localhost:8786")

def unique(filename):
    file, ext = os.path.splitext(filename)
    counter = 0
    while os.path.exists(filename):
        counter += 1
        filename = file + str(counter) + ext
    return filename

tstart = time.time()
output = processor.run_uproot_job(fileset,
                                 treename='Events',
                                 processor_instance=Processor(),
                                 #executor=processor.futures_executor,
                                 executor=processor.dask_executor,
                                 executor_args={'workers':100,"schema": NanoAODSchema, 'client': client,'savemetrics': True},
                                 chunksize = 250000)


[#############################           ] | 73% Completed |  7min  1.4s

In [6]:
dt = time.time() - tstart
outname = 'QCD'
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')
print("Events / s / thread: {:,.0f}".format(output[1]['entries'] / output[1]['processtime']))
print("Events / s: {:,.0f}".format(output[1]['entries'] / dt))

Saving output in histos/QCD.pkl.gz...
Done!


### Plotting
https://github.com/cmstas/FTAnalysis/blob/run2/analysis/fakes/derivation/ScanChain_fast.C#L161

In [7]:
import pickle
path = 'histos/QCD.pkl.gz'
print('Opening path: ', path)
hists = {}
with gzip.open(path) as fin:
    QCD= pickle.load(fin)

Opening path:  histos/QCD.pkl.gz


In [8]:
QCD

{'Muon': <Hist (dataset,pt,eta) instance at 0x7efdda721f70>,
 'Tight_Muon': <Hist (dataset,pt,eta) instance at 0x7efddabf51f0>,
 'Electron': <Hist (dataset,pt,eta) instance at 0x7efddabf5190>,
 'Tight_Electron': <Hist (dataset,pt,eta) instance at 0x7efddabf5400>}

In [32]:
Loose_Muon = Hist2D.from_bincounts(
    QCD[0]['Muon'].sum().values()[('SingleMu',)].T,
    (QCD[0]['Muon'].axis('pt').edges(), QCD[0]['Muon'].axis('eta').edges())
)

In [33]:
Loose_Muon

[[798.00 ± 28.25   297.00 ± 17.23   26142.00 ± 161.68   12972.00 ± 113.89
  2929.00 ± 54.12   455.00 ± 21.33   79.00 ± 8.89]
 [600.00 ± 24.49   269.00 ± 16.40   16356.00 ± 127.89   8385.00 ± 91.57
  1823.00 ± 42.70   309.00 ± 17.58   61.00 ± 7.81]
 [142.00 ± 11.92   65.00 ± 8.06   4347.00 ± 65.93   2231.00 ± 47.23
  450.00 ± 21.21   66.00 ± 8.12   13.00 ± 3.61]]

In [35]:
Tight_Muon = Hist2D.from_bincounts(
    QCD[0]['Tight_Muon'].sum().values()[('SingleMu',)].T,
    (QCD[0]['Tight_Muon'].axis('pt').edges(), QCD[0]['Tight_Muon'].axis('eta').edges())
)

In [36]:
Tight_Muon

[[ 0.00 ± 0.00    0.00 ± 0.00   11258.00 ± 106.10   4867.00 ± 69.76
  909.00 ± 30.15   137.00 ± 11.70   20.00 ± 4.47]
 [ 0.00 ± 0.00    0.00 ± 0.00   8272.00 ± 90.95   3824.00 ± 61.84
  718.00 ± 26.80   110.00 ± 10.49   26.00 ± 5.10]
 [ 0.00 ± 0.00    0.00 ± 0.00   2422.00 ± 49.21   1145.00 ± 33.84
  198.00 ± 14.07   27.00 ± 5.20    8.00 ± 2.83]]

In [42]:
Tight_Muon.divide(Loose_Muon)

[[ 0.00 ± 0.00    0.00 ± 0.00    0.43 ± 0.00    0.38 ± 0.01
   0.31 ± 0.01    0.30 ± 0.03    0.25 ± 0.06]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.51 ± 0.01    0.46 ± 0.01
   0.39 ± 0.02    0.36 ± 0.04    0.43 ± 0.10]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.56 ± 0.01    0.51 ± 0.02
   0.44 ± 0.04    0.41 ± 0.09    0.62 ± 0.28]]